In [18]:
import math
import cv2
import numpy as np
from time import time
import mediapipe as mp
import matplotlib.pyplot as plt
import os
import pandas as pd
os.environ["CUDA_VISIBLE_DEVICES"]="0"

In [19]:
# 미디어 파이프 설정 변수 선언
mp_pose = mp.solutions.pose
pose = mp_pose.Pose(static_image_mode=True,min_detection_confidence=0.3, model_complexity=0)
mp_drawing = mp.solutions.drawing_utils


In [20]:
# 포즈 감지 함수
def detectPose(image,pose,display=True):
    # input 이미지 복사
    output_image = image.copy()
    # RGB로 변환
    imageRGB = cv2.cvtColor(image,cv2.COLOR_BGR2RGB)
    # 렌드마크 찍기
    results = pose.process(imageRGB)
    # 사진 사이즈 구하기
    height, width, _ = image.shape
    # 렌드마크 값 리스트 만들기
    landmarks = []
    visible = []
    if results.pose_landmarks:
        # 렌드마크 그리기
        mp_drawing.draw_landmarks(image=output_image, landmark_list=results.pose_landmarks, connections=mp_pose.POSE_CONNECTIONS)
        #렌드마크 정규값 landmarks 리스트에 넣기
        for landmark in results.pose_landmarks.landmark:
            landmarks.append((int(landmark.x*width), int(landmark.y*height), (landmark.z*width)))
            visible.append(landmark.visibility)
    # 사진 결과 보기
    if display:
        plt.figure(figsize=[22,22])
        plt.subplot(121);plt.imshow(image[:,:,::-1]);plt.title("Original Image");plt.axis('off');
        plt.subplot(122);plt.imshow(output_image[:,:,::-1]);plt.title("Output Image");plt.axis('off');
        # 3차원으로도 봐보기
        mp_drawing.plot_landmarks(results.pose_world_landmarks, mp_pose.POSE_CONNECTIONS)
    else:
        return output_image, landmarks , visible

In [21]:
# 각도 구하는 함수
def calculateAngle(landmark1, landmark2, landmark3):
    x1,y1,_ = landmark1 # z 좌표는 사용 x
    x2,y2,_ = landmark2
    x3,y3,_ = landmark3
    # 3점의 각도 -> 선 2개 -> 각도
    angle = math.degrees(math.atan2(y3-y2, x3-x2) - math.atan2(y1-y2,x1-x2))

    angle = np.abs(angle)
    # 음수의 각도가 나오면 양수로 바꾸기
    if angle >180.0:
        angle = 360 - angle
        
    return angle

In [22]:
# 중앙 각도 구하는 함수
def center_Landmark(landmarks1,landmarks2):
    x1,y1,_ = landmarks1
    x2,y2,_ = landmarks2
    x_cen = (x1+x2)/2
    y_cen = (y1+y2)/2
    return x_cen,y_cen,_

In [26]:
# 사진 한장 -> 3가지 각도를 저장할 프레임 선언 
Datas = pd.DataFrame()
actions = ['bend', 'headup', 'right', 'waistup']
for idx,action in enumerate(actions):
    # # 개수 카운트
    # counter = 0
    # # 처음 상태
    # stage = 'start'
    a = [] #시작각도
    b = [] #고점
    c = [] #1회 끝 (다시 시작 각도)
    df = pd.DataFrame()
    for i in range(len(os.listdir(f'./CNN_dataset/{action}'))):
        # right_pic, headup_pic, leg_pic 바꿔주기 -> 라벨달기 편하게
        image = cv2.imread(f"./CNN_dataset/{action}/{os.listdir(f'./CNN_dataset/{action}')[i]}")
        output_image, landmarks,visible = detectPose(image,pose,display=False)

        if landmarks:
            # 랜드마크 추출 후 각도 계산
            # visible 가져오기
            left_shoulder = visible[mp_pose.PoseLandmark.LEFT_SHOULDER.value]
            right_shoulder = visible[mp_pose.PoseLandmark.RIGHT_SHOULDER.value]
            left_hip = visible[mp_pose.PoseLandmark.LEFT_HIP.value]
            right_hip = visible[mp_pose.PoseLandmark.RIGHT_HIP.value]
            left_knee = visible[mp_pose.PoseLandmark.LEFT_KNEE.value]
            right_knee = visible[mp_pose.PoseLandmark.RIGHT_KNEE.value]
            left_ankle = visible[mp_pose.PoseLandmark.LEFT_ANKLE.value]
            right_ankle = visible[mp_pose.PoseLandmark.RIGHT_ANKLE.value]
            
            # 어깨 - 엉덩이 - 무릎 => 올리는 동작 각도 구하기 위함 -> Main angle
            if left_hip>right_hip:
                main_angle = calculateAngle(landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value],
                                                    landmarks[mp_pose.PoseLandmark.LEFT_HIP.value],
                                                    landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value])
            else:
                main_angle = calculateAngle(landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value],
                                                    landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value],
                                                    landmarks[mp_pose.PoseLandmark.RIGHT_KNEE.value])                                                    
            
            # 엉덩이 - 무릎 - 발목 => 무릎 꺾였는지 보기 위함
            if left_knee > right_knee:
                knee_angle = calculateAngle(landmarks[mp_pose.PoseLandmark.LEFT_HIP.value],
                                                landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value],
                                                landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value])
            else:
                knee_angle = calculateAngle(landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value],
                                                landmarks[mp_pose.PoseLandmark.RIGHT_KNEE.value],
                                                landmarks[mp_pose.PoseLandmark.RIGHT_ANKLE.value])
                                                
            # 어깨 허리  중앙값
            cen_shoulder = center_Landmark(landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value],
                                        landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value])
            cen_hip = center_Landmark(landmarks[mp_pose.PoseLandmark.LEFT_HIP.value],
                                    landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value])
        
            # 코 - 어깨 - 엉덩이 => 목 각도
            head_angle = calculateAngle(landmarks[mp_pose.PoseLandmark.NOSE.value],
                                        cen_shoulder,cen_hip)

            # 엉덩이 중앙의 y좌표와 중앙어깨의 y 좌표사이의 거리 
            waist_len = cen_hip[1]-cen_shoulder[1]

            globals()[f'f{action,i}'] = knee_angle,head_angle,waist_len
            df = df.append([globals()[f'f{action,i}']])
            df['y'] = idx
        # # start -> 각각 리스트 길이가 초기에 0이고 1회 완료시 a,c 의 리스트 길이가 같아짐
        #     if len(a) == 0 or len(a) == len(c) :
        #         if angle >140 : 
        #             print('다리 쭉 펴진 상태 -> 140~180도')
        #             # 이때의 고개 , 무릎 각도 저장
        #             a.append(left_knee_angle)
        #             a.append(left_shoulder_angle)
        #             stage='start'
            
            
        #     # 고점에 도착했을때            
        #     elif len(b) < len(a) :
        #         if angle < 110 :
        #             print('다리 들어 올린상태 -> 110도보다 작아질 정도로 올리기')
        #             b.append(left_knee_angle)
        #             b.append(left_shoulder_angle)
        #             stage = 'up'
            
        #     # 1회 완성 했을 때
        #     elif len(c) < len(b) :
        #         if angle >140 :
        #             print('다시 다시 내린 상태 -> 전과같이 140~180도')
        #             c.append(left_knee_angle)
        #             c.append(left_shoulder_angle)
        #             stage = 'complete'
        #             counter += 1 # 1회 동작 완료
        #             # 동작 완료 후 a,b,c 에 있는 각도 -> 한개의 행으로 합치기 (한개 할때의 각각의 각도가 저장됨)
        #             globals()[f'f{counter}'] = a+b+c
        #             print(globals()[f'f{counter}'])
        #             # 임시 dataframe에 저장
        #             df= df.append([globals()[f'f{counter}']])
        #             # 설정한 action에 정답 데이터 ( 라벨 ) 추가 -> idx,action = enumerate(actions) -> idx(인덱스번호) ㅡ action(내용)
        #             df['y'] = idx
        #             # 다시 abc 리스트 0으로 초기화
        #             a = []
        #             b = []
        #             c = []
    
        # # 횟수 카운트 하는 사각형 만들기
        # cv2.rectangle(output_image, (0,0), (225,73), (245,117,16), -1)

        # # 횟수
        # cv2.putText(output_image, '1set', (15,12), 
        #             cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,0,0), 1, cv2.LINE_AA)

        # cv2.putText(output_image, str(counter), 
        #             (10,60), 
        #         cv2.FONT_HERSHEY_SIMPLEX, 2, (255,255,255), 2, cv2.LINE_AA)
        # # 상태
        # cv2.putText(output_image, 'STAGE', (65,12), 
        #                     cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,0,0), 1, cv2.LINE_AA)
        # cv2.putText(output_image, stage, 
        #                 (60,60), 
        #             cv2.FONT_HERSHEY_SIMPLEX, 2, (255,255,255), 2, cv2.LINE_AA)
        # plt.figure(figsize=[10,10])
        # plt.imshow(output_image[:,:,::-1]);plt.title("Output Image");plt.axis('off');
    Datas = Datas.append(df)

C:\Users\smhrd\AppData\Local\Temp\ipykernel_4060\95283530.py:64: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append([globals()[f'f{action,i}']])
C:\Users\smhrd\AppData\Local\Temp\ipykernel_4060\95283530.py:64: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append([globals()[f'f{action,i}']])
C:\Users\smhrd\AppData\Local\Temp\ipykernel_4060\95283530.py:64: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append([globals()[f'f{action,i}']])
C:\Users\smhrd\AppData\Local\Temp\ipykernel_4060\95283530.py:64: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append([globals()[f'f{action,i}']])
C:\Users\smhrd\AppData\L

In [27]:
column = ['knee_angle','head_angle','waist_length','y']
Datas.columns= column
Datas

,knee_angle,head_angle,waist_length,y
0,113.427073,140.525922,-67.5,0
0,112.676425,140.607560,-67.0,0
0,116.013203,137.897036,-70.5,0
0,105.694821,136.333568,-77.0,0
0,101.624070,134.117542,-78.0,0
...,...,...,...,...
0,173.278882,135.550635,-36.0,3
0,173.278882,135.550635,-36.0,3
0,169.614249,131.191626,-41.5,3
0,170.084159,130.876473,-41.5,3


In [25]:
# csv 파일로 저장하기
Datas.to_csv("03.Knn_Datas.csv",index=None)

In [29]:
Datas

,knee_angle,head_angle,waist_length,y
0,113.427073,140.525922,-67.5,0
0,112.676425,140.607560,-67.0,0
0,116.013203,137.897036,-70.5,0
0,105.694821,136.333568,-77.0,0
0,101.624070,134.117542,-78.0,0
...,...,...,...,...
0,173.278882,135.550635,-36.0,3
0,173.278882,135.550635,-36.0,3
0,169.614249,131.191626,-41.5,3
0,170.084159,130.876473,-41.5,3
